In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

from cc_hardware.drivers.spads import SPADDataType, SPADSensorData
from cc_hardware.drivers.spads.pkl import PklSPADSensorConfig, PklSPADSensor

In [ ]:
pkl_path = Path("../../../logs/20240922_192338/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/10-38-07/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/11-52-34/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/11-54-54/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/13-49-54/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/13-58-09/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/13-59-08/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/18-16-03/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/18-19-19/data.pkl")
assert pkl_path.exists()

sensor = PklSPADSensor(PklSPADSensorConfig.create(pkl_path=pkl_path, loop=True, width=8, height=8, num_bins=18, fovx=45, fovy=45, timing_resolution=250e-12, data_type=SPADDataType.HISTOGRAM | SPADDataType.POINT_CLOUD | SPADDataType.DISTANCE))
height, width = sensor.resolution
print(f"Resolution: {height} x {width}")

index = 10

In [ ]:
histograms = sensor.accumulate(index=index)[SPADDataType.HISTOGRAM]

ylim = None # 1e4
min_bin = 0
max_bin = min(histograms.shape[-1] - 1, 60)

def plot_histogram(i: int, j: int):
    plt.subplot(height, width, i * height + j + 1)
    plt.plot(histograms[i, j])
    # plt.title(f"i={i}, j={j}")
    # plt.yscale("log")

    plt.xlabel("")
    plt.xticks([])
    plt.yticks([])
    plt.ylabel("")

    if ylim is not None:
        plt.ylim(None, ylim)
    if min_bin is not None:
        plt.xlim(min_bin, None)
    if max_bin is not None:
        plt.xlim(None, max_bin)

plt.figure(figsize=(height * 2, width * 2), dpi=400)
for i in range(height):
    for j in range(width):
        plot_histogram(i, j)
plt.tight_layout()

In [ ]:
sensor_data = SPADSensorData(sensor.config)
sensor_data.process(sensor.accumulate(index=index))

data = sensor_data.get_data()
distances = data[SPADDataType.DISTANCE]

pt_cloud = sensor_data.calculate_point_cloud(distances=distances, subpixel_samples=5, bilinear_interpolation=True)

# Flatten the point cloud for Plotly
x = pt_cloud[..., 0].flatten()
y = pt_cloud[..., 1].flatten()
z = pt_cloud[..., 2].flatten()

fig = go.Figure(data=
    [
        go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color='blue', size=3)),
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'  # or 'cube' or 'manual' with 'aspectratio'
    )
)

fig.show()

In [ ]:
all_points = []

for i in range(len(sensor.handler)):
    sensor_data = SPADSensorData(sensor.config)
    sensor_data.process(sensor.accumulate(index=i))
    data = sensor_data.get_data()
    distances = data[SPADDataType.DISTANCE]
    pt_cloud = sensor_data.calculate_point_cloud(distances=distances, subpixel_samples=5, bilinear_interpolation=True)
    all_points.append(pt_cloud.reshape(-1, 3))

combined_points = np.concatenate(all_points, axis=0)
x, y, z = combined_points[:, 0], combined_points[:, 1], combined_points[:, 2]

fig = go.Figure(data=[
    go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color='blue', size=3))
])

fig.update_layout(
    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data')
)

fig.show()


In [ ]:
def backproject(
    voxel_grid: np.ndarray,
    pt_clouds: np.ndarray,
    hists: np.ndarray,
    bin_width: float,
) -> np.ndarray:
    C = 3E8
    thresh = bin_width * C
    factor = bin_width * C / 2
    num_bins = hists.shape[1]

    # Precompute cumulative histograms for all rows to avoid redundant computation.
    cum_hists = np.cumsum(hists, axis=1)

    volume = np.zeros((len(voxel_grid), 1))
    for i, cur_pixel in enumerate(pt_clouds):
        # Compute voxel distances for the current pixel.
        dists = np.linalg.norm(voxel_grid - cur_pixel, axis=1)
        # Retrieve the precomputed cumulative histogram for this pixel.
        cum = cum_hists[i, :]

        # Determine the lower and upper bin indices for each voxel.
        lower = np.clip(
            np.ceil((dists - thresh) / factor).astype(int), 0, num_bins - 1
        )
        upper = np.clip(
            np.floor((dists + thresh) / factor).astype(int), 0, num_bins - 1
        )

        # Compute the range sum using the cumulative histogram.
        sums = cum[upper] - np.where(lower > 0, cum[lower - 1], 0)
        volume += sums.reshape(-1, 1)

    return volume

def filter_volume(volume: np.ndarray, num_x, num_y) -> np.ndarray:
    volume_unpadded = 2 * volume[:, :, 1:-1] - volume[:, :, :-2] - volume[:, :, 2:]
    zero_pad = np.zeros((num_x, num_y, 1))
    volume_padded = np.concatenate([zero_pad, volume_unpadded, zero_pad], axis=-1)
    return volume_padded

def create_voxel_grid(x_range, y_range, z_range, x_res=None, y_res=None, z_res=None, num_x=None, num_y=None, num_z=None):
    if x_res is not None and y_res is not None and z_res is not None:
        num_x = int((x_range[1] - x_range[0]) / x_res)
        num_y = int((y_range[1] - y_range[0]) / y_res)
        num_z = int((z_range[1] - z_range[0]) / z_res)
    elif num_x is not None and num_y is not None and num_z is not None:
        x_res = (x_range[1] - x_range[0]) / num_x
        y_res = (y_range[1] - y_range[0]) / num_y
        z_res = (z_range[1] - z_range[0]) / num_z
    else:
        raise ValueError("Either resolutions (x_res, y_res, z_res) or numbers (num_x, num_y, num_z) must be provided.")

    x = np.linspace(x_range[0], x_range[1], num_x, endpoint=False)
    y = np.linspace(y_range[0], y_range[1], num_y, endpoint=False)
    z = np.linspace(z_range[0], z_range[1], num_z, endpoint=False)
    xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')
    voxel_grid = np.stack([xv.ravel(), yv.ravel(), zv.ravel()], axis=-1)
    return voxel_grid, num_x, num_y, num_z, x_res, y_res, z_res

x_range = [-1, 1]
y_range = [0.0, 1.6]
z_range = [-1.5, -0.5]
num_x = 50
num_y = 50
num_z = 25
voxel_grid, num_x, num_y, num_z, x_res, y_res, z_res = create_voxel_grid(
    x_range=x_range,
    y_range=y_range,
    z_range=z_range,
    num_x=num_x,
    num_y=num_y,
    num_z=num_z,
)

hists_to_use = hists_cropped.copy()
pt_clouds_to_use = pt_clouds.copy()
timestamps_to_use = timestamps.copy()
camera2origin_to_use = camera2origin.copy()
object2origin_to_use = object2origin.copy()

step_size = 20
# If step_size > 1, average the histograms over the step size
if step_size > 1:
    hists_to_use = np.array([
        np.mean(hists_to_use[i:i+step_size], axis=0)
        for i in range(0, len(hists_to_use), step_size)
    ])
    pt_clouds_to_use = np.array([
        np.mean(pt_clouds_to_use[i:i+step_size], axis=0)
        for i in range(0, len(pt_clouds_to_use), step_size)
    ])
    timestamps_to_use = np.array([
        np.mean(timestamps_to_use[i:i+step_size], axis=0)
        for i in range(0, len(timestamps_to_use), step_size)
    ])
    camera2origin_to_use = camera2origin_to_use[::step_size]
    object2origin_to_use = object2origin_to_use[::step_size]

bp_volumes = []
for i in tqdm.tqdm(range(len(pt_clouds_to_use)), desc="Backprojecting"):
    bp_volume = backproject(
        voxel_grid=voxel_grid,
        pt_clouds=pt_clouds_to_use[i],
        hists=hists_to_use[i],
        bin_width=bin_width,
    ).reshape(num_x, num_y, num_z)
    bp_volumes.append(bp_volume)